In [ ]:
import networkx as nx     #python library for the creation, manipulation, functions of complex networks (such as graphs)
                          #used to represent and manipulate the graph, perform graph operations and check connectivity.
import itertools


In [ ]:
def make_all_odd_vertices_even(graph):

    # Create a list of odd-degree vertices
    odd_degree_vertices = [node for node, degree in graph.degree() if degree % 2 == 1]

    # Create pairs of odd-degree vertices
    odd_vertex_pairs = list(itertools.combinations(odd_degree_vertices, 2))

    # Initialize the list of pairs to duplicate the edges between
    pairs_to_duplicate = []
    
#By processing the pairs with the shortest paths first, 
#we attempt to ensure that the most efficient connections are made, 
#and we avoid adding unnecessary long paths to the graph.
    # Find the pair with the shortest path between them
    for pair in odd_vertex_pairs:
        
        # Calculate the shortest path length (weight) between the two vertices in the pair using dijkstra's algorithm provided by NetworkX Library
        path_length = nx.dijkstra_path_length(graph, pair[0], pair[1], weight='weight')

        # Store the pair and its path length in the pairs_to_duplicate list
        pairs_to_duplicate.append((pair, path_length))

    # Sort the pairs by their path length
    pairs_to_duplicate.sort(key=lambda x: x[1])

    # Iterate through the sorted pairs
    for i in range(0, len(pairs_to_duplicate), 2):
        
        # Get the pair with the shortest path
        shortest_pair = pairs_to_duplicate[i][0]

        # Find the shortest path between the vertices in the pair using dijkstra's algorithm provided by NetworkX Library
        shortest_path = nx.dijkstra_path(graph, shortest_pair[0], shortest_pair[1], weight='weight')

        # Duplicate the edges along the shortest path
        for j in range(len(shortest_path) - 1):
            
            u, v = shortest_path[j], shortest_path[j + 1]
            graph.add_edge(u, v, weight=graph[u][v]['weight'])

In [ ]:
# Check if the edge (u, v) is a bridge
def is_bridge(graph, u, v):
    if len(graph[u]) == 1:  # If 'u' has only one neighbor
        return True  # The edge (u, v) is a bridge

    # Store the original edge data (weight) between 'u' and 'v'... 
    #This is done to restore the edge later with its original weight
    original_edge_data = graph[u][v]

    # Remove the edge (u, v) from the graph temporarily to check its impact on graph connectivity
    graph.remove_edge(u, v)

    # Check if the graph is still connected without the edge (u, v)
    # If the graph is connected without the edge (u, v), then it's not a bridge
    # If the graph becomes disconnected, then the edge (u, v) is a bridge
    is_connected = nx.is_connected(graph)

    # Add the edge (u, v) back to the graph with the original edge data (weight)
    #Regardless of whether the graph remains connected or becomes disconnected without the edge (u, v), the edge is always added back to the graph
    graph.add_edge(u, v, **original_edge_data)

    
    # If the graph is not connected without the edge (u, v), it's a bridge
    # so, If the graph remains connected without the edge (u, v), then the edge (u, v) is not a bridge. The is_connected variable is 'True' which means (u, v) is not a bridge and so the function is_bridge() returns 'False'  (not True = False)
    # and for the opposite situation (the edge is a bridge), is_connected will be 'False' and so the function is_bridge() will return 'True'  (not False = True)
    return not is_connected

In [ ]:
# Find the Eulerian circuit using Fleury's Algorithm, considering the modifications made for odd-degree vertices
def fleury_algorithm(graph):
    
    make_all_odd_vertices_even(graph)  # Modify the graph to have all even-degree vertices

    circuit = []  # Initialize the Eulerian circuit as an empty list

    # Start with the first node in the graph and add it to the stack
    stack = [list(graph.nodes())[0]]

    # While there are nodes in the stack
    while stack:
        
        current_vertex = stack[-1]  # Get the last vertex in the stack
        neighbors = list(graph.neighbors(current_vertex))  # Get the neighbors of the current vertex

        # If there are no neighbors for the current vertex
        if not neighbors:
            
            # Add the vertex to the circuit and remove it from the stack
            circuit.append(stack.pop())

        else:  # If there are neighbors for the current vertex
            
            for neighbor in neighbors:
                
                # Check if the edge between the current vertex and the neighbor is not a bridge
                if not is_bridge(graph, current_vertex, neighbor):
                    
                    # If it's not a bridge, break the loop and use this neighbor
                    break
            else:
                # If all edges are bridges, use the first neighbor in the list
                neighbor = neighbors[0]

            # Add the selected neighbor to the stack
            stack.append(neighbor)

            # Remove the traversed edge (current_vertex, neighbor) from the graph
            graph.remove_edge(current_vertex, neighbor)

    # Return the constructed Eulerian circuit
    return circuit